In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install deepspeed==0.6.5
!pip install accelerate
!pip install sentencepiece
!pip install langchain
!pip install torch
!pip install bitsandbytes

In [1]:
import os
# import deepspeed
import torch


In [ ]:
!pip uninstall torch -y
!pip install torch==1.5

In [5]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
4
NVIDIA GeForce GTX 1080 Ti


In [2]:
import torch
print(torch.__version__)


2.0.1+cu117


transformer --> deepseed
accelerate --> deepseed

In [3]:
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline


In [4]:
from langchain.llms import HuggingFacePipeline

In [5]:
import transformers
print(transformers.__version__)

4.30.0


[deeepspeed need torch 1, not install]

accelerate and others need torch 2
transformers==4.30

In [6]:
base_model = LlamaForCausalLM.from_pretrained(
    "chavinlo/alpaca-native",
    load_in_8bit=True,
    device_map='auto',
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
import scipy

66% =>
/dev/sdb2       439G  282G  135G  68% /


In [8]:
tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")

In [8]:
from langchain import PromptTemplate, LLMChain
template = """
Write a SQL Query given the table name {Table} and columns as a list {Columns} for the given question : 
{question}.
"""

prompt = PromptTemplate(template=template, input_variables=["Table","question","Columns"])

In [10]:
from langchain import PromptTemplate, LLMChain
template = """
Write a SQL Query given the table names {Tables} and their respective columns as lists {Columns} for the given question: 
{question}.
"""

prompt = PromptTemplate(template=template, input_variables=["Tables", "question", "Columns"])


In [14]:
pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    max_length=500,
    temperature=0.3,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)
llm_chain = LLMChain(prompt=prompt, llm=local_llm)

In [12]:
def get_llm_response(tble,question,cols):
    llm_chain = LLMChain(prompt=prompt, 
                         llm=local_llm
                         )
    response= llm_chain.run({"Tables" : tble,"question" :question, "Columns" : cols})
    print(response)

In [ ]:
def get_llm_response(tble,question,cols):
    llm_chain = LLMChain(prompt=prompt, 
                         llm=local_llm
                         )
    response= llm_chain.run({"Table" : tble,"question" :question, "Columns" : cols})
    print(response)

In [ ]:
tble = "employee"
cols = ["id","name","date_of_birth","band","manager_id"]
question = "Query the count of employees in band L6 with 239045 as the manager ID"
get_llm_response(tble,question,cols)

In [15]:
Tables = ["users", "orders"]
question = "Find all users who have made an order"
Columns = [["user_id", "username"], ["order_id", "user_id", "product_name"]]
get_llm_response(Tables,question,Columns)

/home/pc-leadtek/joseph/chatsql/myenv/lib/python3.8/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


Answer: SELECT username FROM users INNER JOIN orders ON user_id = order_id;


SELECT users.username 
FROM users 
JOIN orders ON users.user_id = orders.user_id;


In [16]:
question = "請查出訂單編號 12345 的使用者資料"

get_llm_response(Tables,question,Columns)

SELECT u.username FROM users u INNER JOIN orders o ON u.order_id = o.user_id WHERE o.order_number = 12345;


In [16]:
get_llm_response(tble,question,cols)

Answer: SELECT COUNT(*) FROM employee WHERE band='L6' AND manager_id=239045;


In [17]:
question = "請查出管理員ID是23904 並在 band L6 的員工數量"
get_llm_response(tble,question,cols)

SELECT manager_id = 23904 AND count(employee) AS total_in_L6 FROM employee WHERE band = "L6";


In [ ]:
question = "what is the result of 2+100*300?"
get_llm_response(tble,question,cols)